In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import onnxruntime as rt
import onnx
from skl2onnx.common.data_types import FloatTensorType
from skl2onnx import to_onnx
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from skl2onnx import convert_sklearn

In [2]:
# Let's load the dataset
data = pd.read_csv('data/investigation_train_large_checked.csv')

# Let's specify the features and the target
y = data['checked']
X = data.drop(['checked'], axis=1)
X = X.astype(np.float32)

# Let's split the dataset into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [3]:
from sklearn.linear_model import LinearRegression

# Initialize the model
model = LinearRegression()

# Fit the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Convert regression predictions to binary classifications
# Adjust the threshold (e.g., 0.5) based on your problem
y_pred_binary = (y_pred >= 0.5).astype(int)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred_binary)

# Print accuracy
print(f"Accuracy: {accuracy}")


# Calculate metrics
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print metrics
print(f"Mean Squared Error (MSE): {mse}")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"R-squared (R²): {r2}")

Accuracy: 0.8950384615384616
Mean Squared Error (MSE): 0.08310804516077042
Mean Absolute Error (MAE): 0.23069269955158234
R-squared (R²): 0.35776339225444886


In [4]:
# Define equivalent partitions
partitions = [
    {"name": "Children", "condition": lambda df: df['persoon_leeftijd_bij_onderzoek'] < 18},
    {"name": "Adults", "condition": lambda df: (df['persoon_leeftijd_bij_onderzoek'] >= 18) & (df['persoon_leeftijd_bij_onderzoek'] <= 60)},
    {"name": "Seniors", "condition": lambda df: df['persoon_leeftijd_bij_onderzoek'] > 60},
]

In [5]:
# Apply equivalent partitioning
for partition in partitions:
    partition_data = X_test[partition["condition"](X_test)]
    partition_indices = partition_data.index  # Get the indices of the partition
    partition_labels = y_test.loc[partition_indices]  # Get the actual labels for the partition

    if not partition_data.empty:
        # Predictions using the model
        predictions = model.predict(partition_data)

        y_pred_binary = (predictions >= 0.5).astype(int)

        # Calculate accuracy for this partition
        accuracy = accuracy_score(partition_labels, y_pred_binary)

        # Print partition details
        print(f"Partition: {partition['name']}")
        print(f"Number of data points: {len(partition_data)}")
        print(f"Accuracy: {accuracy:.2f}")
        print(f"Predictions: {np.unique(predictions, return_counts=True)}\n")

Partition: Adults
Number of data points: 22256
Accuracy: 0.90
Predictions: (array([-0.6959284 , -0.6682851 , -0.6437522 , ...,  0.79233444,
        0.7978344 ,  0.8761737 ], dtype=float32), array([1, 1, 1, ..., 1, 1, 1], dtype=int64))

Partition: Seniors
Number of data points: 3744
Accuracy: 0.85
Predictions: (array([-0.65246415, -0.62506616, -0.60793835, ...,  0.62865293,
        0.6650887 ,  0.7002379 ], dtype=float32), array([1, 1, 1, ..., 1, 1, 1], dtype=int64))



In [6]:
# Define equivalent partitions
partitions = [
    {"name": "Women", "condition": lambda df: df['persoon_geslacht_vrouw'] == 1},
    {"name": "Men", "condition": lambda df: df['persoon_geslacht_vrouw'] == 0}
]

In [7]:
from sklearn.metrics import confusion_matrix


In [8]:
# Define equivalent partitions number of childrenfrom sklearn.metrics import confusion_matrix

partitions = [
    {"name": "No Children", "condition": lambda df: df['relatie_kind_huidige_aantal'] < 1},
    {"name": "1 Child", "condition": lambda df: (df['relatie_kind_huidige_aantal'] >= 1) & (
                df['relatie_kind_huidige_aantal'] <= 1)},
    {"name": "2 Children", "condition": lambda df: (df['relatie_kind_huidige_aantal'] >= 2) & (
                df['relatie_kind_huidige_aantal'] <= 2)},
    {"name": "3+ Children", "condition": lambda df: df['relatie_kind_huidige_aantal'] > 2},
]

# Apply equivalent partitioning
for partition in partitions:
    partition_data = X_test[partition["condition"](X_test)]
    partition_indices = partition_data.index  # Get the indices of the partition
    partition_labels = y_test.loc[partition_indices]  # Get the actual labels for the partition

    if not partition_data.empty:
        # Predictions using the model
        predictions = model.predict(partition_data)

        y_pred_binary = (predictions >= 0.5).astype(int)

        # Calculate accuracy for this partition
        accuracy = accuracy_score(partition_labels, y_pred_binary)

        # Calculate confusion matrix
        tn, fp, fn, tp = confusion_matrix(partition_labels, y_pred_binary).ravel()

        # Print partition details
        print(f"Partition: {partition['name']}")
        print(f"Number of data points: {len(partition_data)}")
        print(f"Accuracy: {accuracy:.2f}")
        print(f"True Positives (TP): {tp}")
        print(f"False Positives (FP): {fp}")
        print(f"True Negatives (TN): {tn}")
        print(f"False Negatives (FN): {fn}")
        print(f"Predictions: {np.unique(predictions, return_counts=True)}\n")

Partition: No Children
Number of data points: 8903
Accuracy: 0.92
True Positives (TP): 283
False Positives (FP): 0
True Negatives (TN): 7939
False Negatives (FN): 681
Predictions: (array([-0.6959284 , -0.6682851 , -0.65246415, ...,  0.7262802 ,
        0.7271119 ,  0.7470908 ], dtype=float32), array([1, 1, 1, ..., 1, 1, 1], dtype=int64))

Partition: 1 Child
Number of data points: 12754
Accuracy: 0.88
True Positives (TP): 646
False Positives (FP): 1
True Negatives (TN): 10599
False Negatives (FN): 1508
Predictions: (array([-0.6197175, -0.5801923, -0.5530983, ...,  0.7809058,  0.7978344,
        0.8761737], dtype=float32), array([1, 1, 1, ..., 1, 1, 1], dtype=int64))

Partition: 2 Children
Number of data points: 3842
Accuracy: 0.88
True Positives (TP): 271
False Positives (FP): 0
True Negatives (TN): 3092
False Negatives (FN): 479
Predictions: (array([-0.49534053, -0.49128115, -0.4734137 , ...,  0.7599052 ,
        0.77299947,  0.79233444], dtype=float32), array([1, 1, 1, ..., 1, 1, 1], 

In [9]:
from sklearn.linear_model import LinearRegression

# Initialize the model
model = LinearRegression()

# Fit the model
model.fit(X_train, y_train)

LinearRegression()

In [10]:
# Apply equivalent partitioning
for partition in partitions:
    partition_data = X_test[partition["condition"](X_test)]
    partition_indices = partition_data.index  # Get the indices of the partition
    partition_labels = y_test.loc[partition_indices]  # Get the actual labels for the partition

    if not partition_data.empty:
        # Predictions using the model
        predictions = model.predict(partition_data)

        y_pred_binary = (predictions >= 0.5).astype(int)

        # Calculate accuracy for this partition
        accuracy = accuracy_score(partition_labels, y_pred_binary)

        # Print partition details
        print(f"Partition: {partition['name']}")
        print(f"Number of data points: {len(partition_data)}")
        print(f"Accuracy: {accuracy:.2f}")
        print(f"Predictions: {np.unique(predictions, return_counts=True)}\n")

Partition: No Children
Number of data points: 8903
Accuracy: 0.92
Predictions: (array([-0.6959284 , -0.6682851 , -0.65246415, ...,  0.7262802 ,
        0.7271119 ,  0.7470908 ], dtype=float32), array([1, 1, 1, ..., 1, 1, 1], dtype=int64))

Partition: 1 Child
Number of data points: 12754
Accuracy: 0.88
Predictions: (array([-0.6197175, -0.5801923, -0.5530983, ...,  0.7809058,  0.7978344,
        0.8761737], dtype=float32), array([1, 1, 1, ..., 1, 1, 1], dtype=int64))

Partition: 2 Children
Number of data points: 3842
Accuracy: 0.88
Predictions: (array([-0.49534053, -0.49128115, -0.4734137 , ...,  0.7599052 ,
        0.77299947,  0.79233444], dtype=float32), array([1, 1, 1, ..., 1, 1, 1], dtype=int64))

Partition: 3+ Children
Number of data points: 501
Accuracy: 0.88
Predictions: (array([-3.78229260e-01, -3.62459362e-01, -3.53229702e-01, -3.17593843e-01,
       -3.08613777e-01, -3.04985046e-01, -2.89032072e-01, -2.88129210e-01,
       -2.86139667e-01, -2.76764482e-01, -2.64589638e-01, -2.

# MLP model testing 
Here we do the same thing as above, but with an MLP model 

In [11]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

mlp_model = MLPClassifier(hidden_layer_sizes=(16,8),  
                    activation='relu',            
                    solver='adam',                
                    max_iter=10,                
                    random_state=42,
                    verbose=True)
                    
mlp_model.fit(X_train_scaled, y_train)

Iteration 1, loss = 0.24164074
Iteration 2, loss = 0.09848878
Iteration 3, loss = 0.06062212
Iteration 4, loss = 0.04523239
Iteration 5, loss = 0.03655767
Iteration 6, loss = 0.03036991
Iteration 7, loss = 0.02647690
Iteration 8, loss = 0.02318208
Iteration 9, loss = 0.02092893
Iteration 10, loss = 0.01852728


C:\Users\waded\OneDrive\Bureaublad\Research Project\repo\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(hidden_layer_sizes=(16, 8), max_iter=10, random_state=42,
              verbose=True)

In [12]:
# Define equivalent partitions number of childrenfrom sklearn.metrics import confusion_matrix

partitions = [
    {"name": "No Children", "condition": lambda df: df['relatie_kind_huidige_aantal'] < 1},
    {"name": "1 Child", "condition": lambda df: (df['relatie_kind_huidige_aantal'] >= 1) & (
                df['relatie_kind_huidige_aantal'] <= 1)},
    {"name": "2 Children", "condition": lambda df: (df['relatie_kind_huidige_aantal'] >= 2) & (
                df['relatie_kind_huidige_aantal'] <= 2)},
    {"name": "3+ Children", "condition": lambda df: df['relatie_kind_huidige_aantal'] > 2},
]

# Apply equivalent partitioning
for partition in partitions:
    partition_data = X_test[partition["condition"](X_test)]
    partition_indices = partition_data.index  # Get the indices of the partition
    partition_labels = y_test.loc[partition_indices]  # Get the actual labels for the partition

    if not partition_data.empty:
        partition_data_scaled = scaler.transform(partition_data)

        # Predictions using the model
        predictions = mlp_model.predict(partition_data_scaled)

        y_pred_binary = (predictions >= 0.5).astype(int)

        # Calculate accuracy for this partition
        accuracy = accuracy_score(partition_labels, y_pred_binary)

        # Calculate confusion matrix
        tn, fp, fn, tp = confusion_matrix(partition_labels, y_pred_binary).ravel()

        # Print partition details
        print(f"Partition: {partition['name']}")
        print(f"Number of data points: {len(partition_data)}")
        print(f"Accuracy: {accuracy:.2f}")
        print(f"True Positives (TP): {tp}")
        print(f"False Positives (FP): {fp}")
        print(f"True Negatives (TN): {tn}")
        print(f"False Negatives (FN): {fn}")
        print(f"Predictions: {np.unique(predictions, return_counts=True)}\n")

Partition: No Children
Number of data points: 8903
Accuracy: 0.98
True Positives (TP): 877
False Positives (FP): 50
True Negatives (TN): 7889
False Negatives (FN): 87
Predictions: (array([False,  True]), array([7976,  927], dtype=int64))

Partition: 1 Child
Number of data points: 12754
Accuracy: 0.98
True Positives (TP): 2030
False Positives (FP): 116
True Negatives (TN): 10484
False Negatives (FN): 124
Predictions: (array([False,  True]), array([10608,  2146], dtype=int64))

Partition: 2 Children
Number of data points: 3842
Accuracy: 0.98
True Positives (TP): 715
False Positives (FP): 33
True Negatives (TN): 3059
False Negatives (FN): 35
Predictions: (array([False,  True]), array([3094,  748], dtype=int64))

Partition: 3+ Children
Number of data points: 501
Accuracy: 0.98
True Positives (TP): 100
False Positives (FP): 7
True Negatives (TN): 391
False Negatives (FN): 3
Predictions: (array([False,  True]), array([394, 107], dtype=int64))



In [5]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

mlp_model = MLPClassifier(hidden_layer_sizes=(16,8),  
                    activation='relu',            
                    solver='adam',                
                    max_iter=10,                
                    random_state=42,
                    verbose=True)
                    
mlp_model.fit(X_train_scaled, y_train)

Iteration 1, loss = 0.24164074
Iteration 2, loss = 0.09848879
Iteration 3, loss = 0.06062212
Iteration 4, loss = 0.04523239
Iteration 5, loss = 0.03655768
Iteration 6, loss = 0.03036991
Iteration 7, loss = 0.02647690
Iteration 8, loss = 0.02318208
Iteration 9, loss = 0.02092893
Iteration 10, loss = 0.01852728


/Users/luca/Desktop/TUDelft/Y2/DSAIT4015_SETAIS/ai_testing_group_20/aitesting/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(hidden_layer_sizes=(16, 8), max_iter=10, random_state=42,
              verbose=True)

In [6]:
# Define equivalent partitions number of childrenfrom sklearn.metrics import confusion_matrix

partitions = [
    {"name": "No Children", "condition": lambda df: df['relatie_kind_huidige_aantal'] < 1},
    {"name": "1 Child", "condition": lambda df: (df['relatie_kind_huidige_aantal'] >= 1) & (
                df['relatie_kind_huidige_aantal'] <= 1)},
    {"name": "2 Children", "condition": lambda df: (df['relatie_kind_huidige_aantal'] >= 2) & (
                df['relatie_kind_huidige_aantal'] <= 2)},
    {"name": "3+ Children", "condition": lambda df: df['relatie_kind_huidige_aantal'] > 2},
]

# Apply equivalent partitioning
for partition in partitions:
    partition_data = X_test[partition["condition"](X_test)]
    partition_indices = partition_data.index  # Get the indices of the partition
    partition_labels = y_test.loc[partition_indices]  # Get the actual labels for the partition

    if not partition_data.empty:
        partition_data_scaled = scaler.transform(partition_data)

        # Predictions using the model
        predictions = mlp_model.predict(partition_data_scaled)

        y_pred_binary = (predictions >= 0.5).astype(int)

        # Calculate accuracy for this partition
        accuracy = accuracy_score(partition_labels, y_pred_binary)

        # Calculate confusion matrix
        tn, fp, fn, tp = confusion_matrix(partition_labels, y_pred_binary).ravel()

        # Print partition details
        print(f"Partition: {partition['name']}")
        print(f"Number of data points: {len(partition_data)}")
        print(f"Accuracy: {accuracy:.2f}")
        print(f"True Positives (TP): {tp}")
        print(f"False Positives (FP): {fp}")
        print(f"True Negatives (TN): {tn}")
        print(f"False Negatives (FN): {fn}")
        print(f"Predictions: {np.unique(predictions, return_counts=True)}\n")

Partition: No Children
Number of data points: 8903
Accuracy: 0.98
True Positives (TP): 877
False Positives (FP): 50
True Negatives (TN): 7889
False Negatives (FN): 87
Predictions: (array([False,  True]), array([7976,  927]))

Partition: 1 Child
Number of data points: 12754
Accuracy: 0.98
True Positives (TP): 2030
False Positives (FP): 116
True Negatives (TN): 10484
False Negatives (FN): 124
Predictions: (array([False,  True]), array([10608,  2146]))

Partition: 2 Children
Number of data points: 3842
Accuracy: 0.98
True Positives (TP): 715
False Positives (FP): 33
True Negatives (TN): 3059
False Negatives (FN): 35
Predictions: (array([False,  True]), array([3094,  748]))

Partition: 3+ Children
Number of data points: 501
Accuracy: 0.98
True Positives (TP): 100
False Positives (FP): 7
True Negatives (TN): 391
False Negatives (FN): 3
Predictions: (array([False,  True]), array([394, 107]))

